In [1]:
%useLatestDescriptors
%use koog(v = 0.5.1)
import kotlinx.coroutines.runBlocking













































In [2]:
val apiKey = System.getenv("OPEN_ROUTER_API_KEY")
val executor = simpleOpenRouterExecutor(apiKey)

In [3]:
val agent = AIAgent(
    promptExecutor = executor,
    llmModel = OpenRouterModels.GPT_OSS_120b,
    systemPrompt = "You are a travel agent. You need to provide the best route for the user."
)

In [4]:
runBlocking {
    agent.run("What is Ktlin? Tell me one sentence about it.")
}

Kotlin is a modern, statically‑typed programming language that runs on the JVM (and can also compile to JavaScript and native code).  

If you need help planning a trip or finding the best route somewhere, just let me know the details and I’ll gladly assist!

In [18]:
import ai.koog.agents.core.tools.annotations.Tool
import kotlinx.serialization.Serializable

@LLMDescription("Tools for getting weather information")
class MyFirstToolSet : ToolSet {
    @Tool
    @LLMDescription("Get the current weather for a location")
    fun getWeather(
        @LLMDescription("The city and state/country")
        location: String
    ): String {
        // In a real implementation, you would call a weather API
        return "The weather in $location is sunny and 72°F"
    }
}

@Serializable
@LLMDescription("response")
class Response(
    @Serializable
    @property:LLMDescription("summary")
    val summary: String,
    @Serializable
    @property:LLMDescription("details")
    val details: String,
)

val customStrategy = strategy<String, String>("custom-strategy") {
    val subGraphWithTask by subgraphWithTask<String, Response>(tools = MyFirstToolSet().asTools()) { input ->
        input
    }

    edge(nodeStart forwardTo subGraphWithTask)
    edge(
        subGraphWithTask forwardTo nodeFinish
                transformed { "summary: ${it.summary}\ndetails: ${it.details}" }
    )
}

val agentWithCustomStrategy = AIAgent(
    promptExecutor = executor,
    llmModel = OpenRouterModels.Gemini2_5Flash,
    systemPrompt = "You are helpful agent. You need to call tool, when needed.",
    strategy = customStrategy,
    toolRegistry = ToolRegistry {
        tools(MyFirstToolSet())
    }
)

runBlocking {
    agentWithCustomStrategy.run("Tell me the weather in Seattle.")
}

summary: Weather in Seattle
details: The weather in Seattle, WA is sunny and 72°F